In [1]:
import os
import imblearn
import numpy as np
import pandas as pd
from math import sqrt
import xgboost as xgb
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from imblearn.pipeline import Pipeline
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import SGDRegressor
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import f1_score, classification_report, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.feature_selection import f_regression, f_classif, RFE, VarianceThreshold, chi2, SelectKBest, SelectFromModel

## **Data preparation**

In [2]:
# import data and remove unwanted columns
data = pd.read_excel('augmented sample.xlsx').drop(['index',
                                                    'Formation_energy'],
                                                    axis=1)
data = data.set_index(['Material Composition'])

In [3]:
# split the dataset into X and y(Energy Above Hull)
X = data.drop(['EnergyAboveHull'], axis=1)
y = data['EnergyAboveHull']

In [4]:
# preview the shape of the dataframe
sample_size = X.shape[0]
feature_size = X.shape[1]
print("Number of samples", sample_size,
      "\nNumber of features:", feature_size)

Number of samples 2138 
Number of features: 962


In [5]:
# get train and test data
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, 
                                                          test_size=0.2, 
                                                          random_state=42)

In [6]:
# preprocess the y
y_clf = np.zeros_like(y)
y_trainval_clf = np.zeros_like(y_trainval)
y_test_clf = np.zeros_like(y_test)

# samples with EnergyAboveHull larger than 40 will be marked as unstable
# unstable = 0, stable = 1
y_clf = [1*(EAH<=40) for EAH in y]
y_trainval_clf = [1*(EAH<=40) for EAH in y_trainval]
y_test_clf = [1*(EAH<=40) for EAH in y_test]

**for classification:**

In [7]:
print("Number of training samples for classification: ", 
      X_trainval.shape[0])
print("Number of training labels for classification: ", 
      len(y_trainval_clf))
print("Number of test samples for classification: ", 
      X_test.shape[0])
print("Number of test labels for classification: ", 
      len(y_test_clf))

Number of training samples for classification:  1710
Number of training labels for classification:  1710
Number of test samples for classification:  428
Number of test labels for classification:  428


**for regression:**

In [8]:
print("Number of training samples for regression: ", 
      X_trainval.shape[0])
print("Number of training labels for regression: ", 
      len(y_trainval))
print("Number of test samples for regression: ", 
      X_test.shape[0])
print("Number of test labels for regression: ", 
      len(y_test))

Number of training samples for regression:  1710
Number of training labels for regression:  1710
Number of test samples for regression:  428
Number of test labels for regression:  428


## **XGB Regression**

**best parameters**

{'kbest__k': 250, 'model__colsample_bytree': 0.6, 'model__max_depth': 5, 'model__n_estimators': 150, 'pca__n_components': 25}

In [102]:
# best model construction
model = xgb.XGBRegressor(colsample_bytree=0.6, max_depth=5, n_estimators=150)

In [103]:
# best pipeline construction
pipeline = Pipeline([
    ('variance threshold', VarianceThreshold()),
    ('kbest', SelectKBest(f_regression, k=250)),
    ('standard_scaler', StandardScaler()), 
    ('pca', PCA(n_components=25)), 
    ('model', model)
])

In [104]:
fold = 10
avg_mae = []
avg_rmse = []
for i in range(fold):
    X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=(1/fold))
    pipeline.fit(X_trainval, y_trainval)
    
    y_pred = pipeline.predict(X_test)
    y_true = y_test
    
    avg_mae.append(mean_absolute_error(y_true, y_pred))
    avg_rmse.append(sqrt(mean_squared_error(y_true, y_pred)))

In [105]:
print('average mae', 
      '{:.3f}'.format(np.mean(avg_mae)), 
      '±', 
      '{:.3f}'.format(2*np.std(avg_mae)))

average mae 27.484 ± 4.608


In [106]:
print('average rmse', 
      '{:.3f}'.format(np.mean(avg_rmse)), 
      '±', 
      '{:.3f}'.format(2*np.std(avg_rmse)))

average rmse 47.229 ± 27.223


## **XGB Classification**

**best parameters**

{'kbest__k': 450, 'model__colsample_bytree': 0.6, 'model__max_depth': 5, 'model__n_estimators': 650, 'pca__n_components': 50}

{'sampling__k_neighbors': 1, 'sampling__m_neighbors': 1}

In [58]:
# best model construction
model = xgb.XGBClassifier(max_depth=5, 
                          n_estimators=650, 
                          colsample_bytree=0.6)

In [59]:
# best pipeline construction
pipeline = Pipeline([
    ('sampling', BorderlineSMOTE(k_neighbors=1, 
                                 m_neighbors=1, 
                                 sampling_strategy='minority')),
    ('variance threshold', VarianceThreshold()),
    ('kbest', SelectKBest(f_classif, k=450)),
    ('standard_scaler', StandardScaler()), 
    ('pca', PCA(n_components=50)), 
    ('model', model)
])

In [61]:
fold = 10
weighted_avg_f1 = []
for i in range(fold):
    X_trainval, X_test, y_trainval, y_test = train_test_split(X, y_clf, test_size=(1/fold))
    pipeline.fit(X_trainval, y_trainval)
    
    y_pred = pipeline.predict(X_test)
    y_true = y_test
    
    target_names = ['unstale', 'stable']
    report = classification_report(y_true, y_pred, target_names=target_names, output_dict=True)
    weighted_avg_f1.append(report['weighted avg']['f1-score'])

In [93]:
print('average weighted f1 score', 
      '{:.3f}'.format(np.mean(weighted_avg_f1)), 
      '±', 
      '{:.3f}'.format(2*np.std(weighted_avg_f1)))

average weighted f1 score 0.919 ± 0.020
